# CSC_5RO15_TA_TP1: Direct and Inverse Kinematics

## Introduction

We propose to study the __geometric__ and __kinematic__ modeling of a manipulator arm developed by the *Interactive Robotics Laboratory* of the *CEA List*.

This robot, which kinematic chain is of serial type, has $6$ revolute joints ($j_i$ with $i=1,\ldots,6$).

<img 
	style="display: block; 
		margin-left: auto;
		margin-right: auto;"
	src="Images/VueRobot.png"
	width="400">
</img>

The numerical values of the robot parameters, required for the completion of this tutorial, are specified in the following table.

$$
\begin{aligned}
& \text {Table. Numerical values of the robot parameters.}\\
	&\begin{array}{cccc}
	\hline
		\text{Parameters} &	\text{Numerical values} &	\text{Type of parameter} \\ 
		\hline
		d_3 & 0.7 m & \text{Geometric parameter} \\
		r_1 & 0.5 m & \text{Geometric parameter} \\
		r_4 & 0.2 m & \text{Geometric parameter} \\
		r_E & 0.1 m & \text{Geometric parameter} \\
	\hline
	\end{array}
\end{aligned}
$$

The use of *Python* is required to perform the tutorial. Please import the following required mathematical libraires to start the tutorial.

In [1]:
from numpy.linalg import eig

import functools as fu
import math as m
import matplotlib.pyplot as plt
import numpy as np
import random

In the following, you will progressively update a *Dictionnary* in Python containing the robot parameters, named __robotParameters__.

Please initialize it as follows: 
*robotParameters = { 'nJoints': 6, 'jointsType': ['R','R','R','R','R','R']}*

In [2]:
robotParameters = { 'nJoints': 6, 'jointsType': ['R','R','R','R','R','R']}

You will also progressively build a *Class* containing some *attributes* related to the robot. To do so, you will be asked to program some of its *methods* in the tutorial. This class is named __RobotModel__ and is defined in the file *ClassRobotModel*.

Please initialize it as follows. You will see printing the resulting *attributes* of the *Class* __RobotModel__.

In [3]:
from ClassRobotModel import RobotModel
RobotTutorials = RobotModel( **robotParameters )

Attribute (int): self.numberJoints =  6
Attribute (list): self.jointsType =  ['R', 'R', 'R', 'R', 'R', 'R']
Attribute (list - 0 if self.jointsType[i] == 'R' / 1 if self.jointsType[i] == 'P'): self.sigma =  [0, 0, 0, 0, 0, 0]


## DGM, Direct Geometric Model

### Q1

Fill in the following figure giving the frames attached to the successive links of the robot according to the *Modified Denavit-Hartenberg (MDH)* parameters defining the spatial arrangement of the robot structure (axis names and geometric distances should be reported on the completed figure).

<img
    style=  "display: block;
            margin-left: auto;
            margin-right: auto;"
    src="Images/ParametresDH.png"
    width="600">
</img>

From pages 93 and 94 of the Interactive Robotic Systems book the references according to the *Modified Denavit-Hartenberg (MDH)* are presented below:

<img
    style=  "display: block;
            margin-left: auto;
            margin-right: auto;"
    src="Images/ParametresDH_updated.png"
    width="600">
</img>

### Q2.

To fill the following table with the geometric parameters of the robot:

$$
\begin{array}{ccccc}
		\hline
		\hline
		 i  & \alpha_i & d_i & \theta_i & r_i \\
		\hline
		\hline
		1  & ? & ? & ? & ? \\
		\hline
		2  & ? & ? & ? & ? \\
		\hline
		3  & ? & ? & ? & ? \\
		\hline
		4  & ? & ? & ? & ? \\
		\hline
		5  & ? & ? & ? & ? \\
		\hline
		6  & ? & ? & ? & ? \\
		\hline
		\hline
\end{array}
$$

please complete the file named *DHM_parameters.txt* in the current repository.

From page 95 the Interactive Robotic Systems book the geometric parameters of the robot are presented below

$$
\begin{array}{ccccc}
		\hline
		\hline
		 i  & \alpha_i & d_i & \theta_i & r_i \\
		\hline
		\hline
		1  & 0 & 0 & 0 & 0.5 \\
		\hline
		2  & +\pi/2 & 0 & 0 & 0 \\
		\hline
		3  & 0 & 0.7 & +\pi/2 & 0 \\
		\hline
		4  & +\pi/2 & 0 & 0 & 0.2 \\
		\hline
		5  & -\pi/2 & 0 & 0 & 0 \\
		\hline
		6  & +\pi/2 & 0 & 0 & 0 \\
		\hline
		\hline
\end{array}
$$

Then, update the dictionnary __robotParameters__ with the DHM parameters as follows: 

*robotParameters['fileDHM'] = "DHM_parameters.txt"*

*RobotTutorials = ClassRobotModel.RobotModel( **robotParameters )*

Look at the resulting *attribute* named *"self.tableDHM"*

In [4]:
robotParameters['fileDHM'] = "DHM_parameters.txt"
RobotTutorials = RobotModel( **robotParameters )

Attribute (int): self.numberJoints =  6
Attribute (list): self.jointsType =  ['R', 'R', 'R', 'R', 'R', 'R']
Attribute (list - 0 if self.jointsType[i] == 'R' / 1 if self.jointsType[i] == 'P'): self.sigma =  [0, 0, 0, 0, 0, 0]
Attribute (list - float): self.tableDHM =  [[1.0, 0.0, 0.0, 0.0, 0.5], [2.0, 1.5707963267948966, 0.0, 0.0, 0.0], [3.0, 0.0, 0.7, 1.5707963267948966, 0.0], [4.0, 1.5707963267948966, 0.0, 0.0, 0.2], [5.0, -1.5707963267948966, 0.0, 0.0, 0.0], [6.0, 1.5707963267948966, 0.0, 0.0, 0.0]]


### Q3.

#### Q3a.

Write a generic function $TransformMatElem(\alpha_i, d_i, \theta_i, r_i)$ which output argument is the homogeneous transform matrix *g* between two successive frames.

In [5]:
@staticmethod
def TransformMatElem(alpha_i, d_i, theta_i, r_i):
    """ 
    Computation of the homogeneous transform matrix between two successive frames R_(i-1) and R_i
    
    Input: 
        - Four scalar parameters given by the Modified Denavit-Hartenberg (MDH) convention
         
    Output: 
        - Homogeneous transform matrix g_(i-1,i) as a "np.array"
    """
    
    x = np.array([
            [+np.cos(theta_i), -np.sin(theta_i), 0.0, d_i],
            [+np.cos(alpha_i)*np.sin(theta_i), +np.cos(alpha_i)*np.cos(theta_i), -np.sin(alpha_i), -r_i*np.sin(alpha_i)],
            [+np.sin(alpha_i)*np.sin(theta_i), +np.sin(alpha_i)*np.cos(theta_i), +np.cos(alpha_i), +r_i*np.cos(alpha_i)],
            [0.0, 0.0, 0.0, 1.0],
        ]
    )

    return x

RobotModel.TransformMatElem = TransformMatElem

From page 96 of the Interactive Robotic Systems book the function above was implemented.

#### Q3b.

Write a function $ComputeDGM(\text{self}, q)$ which computes the direct geometric model of any robot with series open kinematic chain, taking as input arguments the current configuration.

In [6]:
def ComputeDGM(self, q_cur):
   """
   Computation of the Direct Geometric Model (DGM) of the robot given by its MDH parameters for the joint configuration q_cur
   
   Inputs: 
      - List of robot's geometric parameters "self.tableDHM" given by the Modified Denavit-Hartenberg (MDH) convention
      - Number of joints of the robot "self.numberJoints"
      - List of type of joints of the robot: "self.sigma"
      - Current joint configuration "q_cur"
      
   Outputs: 
      - List of the successive homogeneous transform matrices: "self.list_g_i_1_i = [g_01, ..., g_N_1_N]"
      - List of the successive resulting homogeneous transform matrices: "self.list_g_0i = [g_01, g_02, ..., g_ON]"
   """

   self.list_g_i_1_i = []


   for i, alpha, d, theta, r in self.tableDHM:
      self.list_g_i_1_i.append(self.TransformMatElem(alpha, d, theta + q_cur[int(i)-1], r))


   self.list_g_0i = [self.list_g_i_1_i[0]]

   for g_i in self.list_g_i_1_i[1:]:
      self.list_g_0i.append(self.list_g_0i[-1] @ g_i)

RobotModel.ComputeDGM = ComputeDGM

From page 97 of the Interactive Robotic Systems book the function above was implemented.

#### Q3c.

We consider an end-effector mounted at the end of the robot arm. The frame $\mathcal{R}_E$ attached to the end-effector of the robot is defined by a translation of the frame  $\mathcal{R}_6$ by a distance $r_E$ along the $z_6$ axis.

Specify the four DHM parameters for the tool frame description in the field below.


In [7]:
robotParameters['toolFrameDHM'] = [ 0, 0, 0, 0.1 ]
RobotTutorials = RobotModel( **robotParameters )

Attribute (int): self.numberJoints =  6
Attribute (list): self.jointsType =  ['R', 'R', 'R', 'R', 'R', 'R']
Attribute (list - 0 if self.jointsType[i] == 'R' / 1 if self.jointsType[i] == 'P'): self.sigma =  [0, 0, 0, 0, 0, 0]
Attribute (list - float): self.tableDHM =  [[1.0, 0.0, 0.0, 0.0, 0.5], [2.0, 1.5707963267948966, 0.0, 0.0, 0.0], [3.0, 0.0, 0.7, 1.5707963267948966, 0.0], [4.0, 1.5707963267948966, 0.0, 0.0, 0.2], [5.0, -1.5707963267948966, 0.0, 0.0, 0.0], [6.0, 1.5707963267948966, 0.0, 0.0, 0.0]]
Attribute (list - float): self.toolDHM =  [0, 0, 0, 0.1]


Using the results of previous questions, write a function $ComputeToolPose(\text{self})$  that computes the homogeneous transform matrix $\overline{g}_{0E}$. This matrix gives the position and the orientation of the frame $\mathcal{R}_E$ attached to the end-effector of the robot, expressed in the base frame $\mathcal{R}_0$.

In [8]:
def ComputeToolPose(self):
   """ 
   Computation of the homogeneous transform matrix g0E which gives the position and the orientation of the frame R_E attached to the end-effector of the robot, expressed in the base frame R_0 evaluated at the joint configuration q_cur

   Inputs:
      - List of the successive homogeneous transform matrices "self.list_g_0i"
      - Number of joints of the robot "self.numberJoints"
      - List of the geometric parameters of the tool "self.toolDHM" given by the Modified Denavit-Hartenberg (MDH) convention

   Output:
      - Homogeneous transform matrix "self.g_0E"
   """

   self.g_0E = self.list_g_0i[-1] @ self.TransformMatElem(self.toolDHM[0], self.toolDHM[1], self.toolDHM[2], self.toolDHM[3])

RobotModel.ComputeToolPose = ComputeToolPose

From page 97 of the Interactive Robotic Systems book the function above was implemented.

In the following, we consider two joint configurations $q=\left[q_1, \ldots, q_6\right]^t$ of the robot: $q_i=\left[-\frac{\pi}{2}, 0 , -\frac{\pi}{2} , -\frac{\pi}{2} , -\frac{\pi}{2} , -\frac{\pi}{2} \right]^t$ and $q_f=\left[0, \frac{\pi}{4}  , 0 , \frac{\pi}{2} , \frac{\pi}{2} , 0\right]^t$.

Indicate what are the homogeneous transform matrices $\overline{g}_{0E}$ evaluated in these two confugrations.

In [13]:
q_i = [-np.pi/2, 0, -np.pi/2, -np.pi/2, -np.pi/2, -np.pi/2]
q_f = [0, np.pi/4, 0, np.pi/2, np.pi/2, 0]

In [14]:
RobotTutorials.ComputeDGM(q_i)
RobotTutorials.ComputeToolPose()
print(RobotTutorials.g_0E)

print()

RobotTutorials.ComputeDGM(q_f)
RobotTutorials.ComputeToolPose()
print(RobotTutorials.g_0E)

[[-6.12323400e-17 -6.12323400e-17 -1.00000000e+00 -1.00000000e-01]
 [ 1.00000000e+00 -1.23259516e-32 -6.12323400e-17 -7.00000000e-01]
 [-1.23259516e-32 -1.00000000e+00  6.12323400e-17  3.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

[[-7.07106781e-01  7.07106781e-01  3.01573569e-33  6.36396103e-01]
 [-4.32978028e-17 -4.32978028e-17 -1.00000000e+00 -1.00000000e-01]
 [-7.07106781e-01 -7.07106781e-01  6.12323400e-17  1.13639610e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


### Q4.

What are the values of positions $P_x, P_y, P_z$ and the parameters related to the orientation $R_{n,q}$ ($n$ being the direction vector and $q \in \left[0,\pi\right]$ the rotation angle such that $R_{n,q}=R_{0E}$) of the end-effector frame for the two joint configurations $q_i=\left[-\frac{\pi}{2}, 0 , -\frac{\pi}{2} , -\frac{\pi}{2} , -\frac{\pi}{2} , -\frac{\pi}{2} \right]^t$ and $q_f=\left[0, \frac{\pi}{4}  , 0 , \frac{\pi}{2} , \frac{\pi}{2} , 0\right]^t$ ($q=\left[q_1, \ldots, q_6\right]^t$)?
To do so, write a function $DescribeToolFrame(\text{self})$ that computes the position vector and the parameters related to the orientation of the end-effector frame for the current configuration.

In [ ]:
def DescribeToolFrame(self):
        """
        Computation of the position vector and the parameters related to the orientation R_n,q of the end-effector frame for the configuration q as specified in the constructor function

        Input: 
                - Direct Geometric Model (DGM) of the robot including its end-effector through "self.g_0E"

        Outputs: 
                - Values of positions P=[Px, Py, Pz]' (in m) of the origin of frame R_E w.r.t. R_0 given in R_0: "self.P"
                - Orientation parameters R_n,q, as follows:
                        - "self.n": being the direction vector
                        - "self.q" in [0,pi] the rotation angle in rad such that R_n,q = R_0E
        """

        R = self.g_0E[0:3,0:3] # rotation matrix
        r11, r12, r13 = R[0]
        r21, r22, r23 = R[1]
        r31, r32, r33 = R[2]

        # book notation
        theta = np.atan2(1/2 * np.sqrt( (r32 - r23)**2 + (r13 - r31)**2 + (r21 - r12)**2 ), 1/2 * (r11 + r22 + r33 - 1))

        if theta != 0:
                w = [(r32 - r23)/(2 * theta), (r13 - r31)/(2 * theta), (r21 - r12)/(2 * theta)]
        else:
                w = [0, 0, 0]

        # Position vector
        P = self.g_0E[0:3, 3]

        self.P = P
        self.n = w
        self.q = theta


RobotModel.DescribeToolFrame = DescribeToolFrame

In [17]:
RobotTutorials.ComputeDGM(q_i)
RobotTutorials.ComputeToolPose()
RobotTutorials.DescribeToolFrame()
print(RobotTutorials.P, RobotTutorials.n, RobotTutorials.q)

print()

RobotTutorials.ComputeDGM(q_f)
RobotTutorials.ComputeToolPose()
RobotTutorials.DescribeToolFrame()
print(RobotTutorials.P, RobotTutorials.n, RobotTutorials.q)

[-0.1 -0.7  0.3] [np.float64(-0.23873241463784295), np.float64(-0.23873241463784298), np.float64(0.23873241463784298)] 2.0943951023931957

[ 0.6363961 -0.1        1.1363961] [np.float64(0.05646538721156071), np.float64(0.1363195036107982), np.float64(-0.1363195036107982)] 2.5935642459694805


From pages 60 and 61 of the Interactive Robotic Systems book the function above was implemented.

## DKM, Direct Kinematic Model

### Q5.

Write a function $ComputeJac\left(self, q\right)$ which output is the Jacobian matrix ${}^0J(q)$ (computed by the method of velocities composition).

Reminder: the Jacobian matrix relates the velocities in the task coordinates of the end-effector frame in $\mathcal{R}_0$, for a given joint configuration $q$, to the joint velocities:
\begin{align*}
{}^0\mathcal{V}_{0,E}&= \begin{bmatrix}^0V_{0,E}\left(O_E\right) \\ ^0\omega_{0,E} \end{bmatrix} = \begin{bmatrix} {}^0J_v \left(q\right)  \\ {}^0J_{\omega} \left(q\right)\end{bmatrix} \dot{q} = {}^0J\left(q\right) \dot{q}
\end{align*}

In [27]:
def ComputeJac(self, q_cur):
   """ 
   Computation of the Jacobian matrix mapping the joint velocities to the velocities of the end-effector for a joint configuration "q_cur"

   Inputs: 
      - List defining the types of joints : "self.jointsType" 
      - Number of joints of the robot: "self.numberJoints"
      - Current configuration "q_cur"
   
   Output: 
      - Jacobian matrix 0_J in R_0: "self.oJ" as np.array
   """

   self.ComputeDGM(q_cur)

   self.oJ = np.zeros((6, self.numberJoints))

   for i in range(self.numberJoints):
      if self.jointsType[i] == 'R':
         self.oJ[0:3, i] = np.cross(self.list_g_0i[-1][0:3, 2], self.list_g_0i[-1][0:3, 3] - self.list_g_0i[i][0:3, 3])
         self.oJ[3:6, i] = self.list_g_0i[-1][0:3, 2]

      elif self.jointsType[i] == 'P':
         self.oJ[0:3, i] = self.list_g_0i[-1][0:3, 2]
         self.oJ[3:6, i] = np.zeros(3)

RobotModel.ComputeJac = ComputeJac

What are the values of the twists at $O_E$ evaluated with $q=q_i$ and $q=q_f$ with the joint velocities $\dot q = \left[0.5, 1.0, -0.5, 0.5, 1.0, -0.5\right]^t$?

In [28]:
q_dot = np.array(
    [+0.5, +1.0, -0.5, +0.5, +1.0, -0.5]
).T

print(q_dot)

[ 0.5  1.  -0.5  0.5  1.  -0.5]


In [29]:
RobotTutorials.ComputeJac(q_i)
print(RobotTutorials.oJ @ q_dot)

print()

RobotTutorials.ComputeJac(q_f)
print(RobotTutorials.oJ @ q_dot)

[ 7.65404249e-17 -2.00000000e-01  1.05000000e+00 -2.00000000e+00
 -1.22464680e-16  1.22464680e-16]

[-8.83883476e-01  5.41222535e-17  8.83883476e-01  6.03147138e-33
 -2.00000000e+00  1.22464680e-16]


### Q6.

In the rest of the study, we restrict the analysis of operational end-effector velocities to translational velocities via ${}^0J_v \left(q\right)$.

Qualify the transmission of velocities between the joint and task spaces for the corresponding $q_i$ and $q_f$ configurations: what is the preferred direction to transmit velocity in the task space when the manipulator configuration is $q_i$? Same question for $q_f$? What are the corresponding velocity manipulabilities? 
To help, you can program a function $QualifyVelocityTransmission(self)$ that analyses the property of the Jacobian matrix. Explain your results.

In [34]:
def QualifyVelocityTransmission(self):
   """ 
   Qualifying the transmission of velocities

   Input:
      - Jacobian matrix "self.oJ" to be analysed
   
   Outputs:
   """

   # Compute the singular values of the Jacobian matrix
   s = np.linalg.svd(self.oJ, compute_uv=False)

   # Compute the condition number

   self.conditionNumber = np.max(s) / np.min(s) if np.min(s) > 1e-6 else np.inf

   # Compute the rank of the Jacobian matrix
   self.rank = np.linalg.matrix_rank(self.oJ)

   # Compute the determinant of the Jacobian matrix
   self.det = np.linalg.det(self.oJ)


RobotModel.QualifyVelocityTransmission = QualifyVelocityTransmission

In [35]:
RobotTutorials.ComputeJac(q_i)
RobotTutorials.QualifyVelocityTransmission()
print(RobotTutorials.conditionNumber, RobotTutorials.rank, RobotTutorials.det)

print()

RobotTutorials.ComputeJac(q_f)
RobotTutorials.QualifyVelocityTransmission()
print(RobotTutorials.conditionNumber, RobotTutorials.rank, RobotTutorials.det)

inf 3 0.0

inf 2 0.0


## IGM, Inverse Geometric Model

### Q7.

In this study, the resolution of the inverse geometric model is considered numerically by exploiting the inverse differential model. Moreover, the study is restricted to the position only of the robot's end-effector frame in the task space (no constraint on the orientation of the end-effector frame).

Using an iterative procedure exploiting the pseudo-inverse of the Jacobian matrix, program a function $ComputeIGM(\text{self}, X_d, q_0, k_{max}, \epsilon_x)$ having as input arguments the desired task position $X_d$ and the initial condition $q_0$. Both the maximum number of iterations $k_{max}$ of the algorithm and the norm of the tolerated Cartesian error $|X_d - DGM(q_{k})|<\epsilon_x$, define the stopping criteria of the algorithm.

In [49]:
def ComputeIGM(self, X_d, q_0, k_max, eps_x):
         """
         Computation of the Inverse Geometric Model (IGM) mapping the Cartesian pose to the joint vector "q"

         Inputs:
            - Desired Cartesian vector "X_d" as a np.array to be reached by the robot 
            - Initial condition "q_0" as a np.array
            - Number "k_max" of maximal iteration in the recursive algorithm 
            - Norm of the tolerated Cartesian error "eps_x"

         Outputs:
            - List "self.list_q_IGM" of the joint vectors computed at each iteration of the recursive algorithm 
            - Returned "self.list_q_IGM[-1]" of the final found joint vector, solution of the IGM
            
         """

         self.list_q_IGM = [q_0]
         q_k = q_0

         for _ in range(k_max):
            self.ComputeDGM(q_k)
            X_k = self.g_0E[0:3, 3]
            epsilon_x = X_d - X_k

            if np.linalg.norm(epsilon_x) < eps_x:
               break

            self.ComputeJac(q_k)
            J_v = self.oJ[0:3, :]
            q_k = q_k + np.linalg.pinv(J_v) @ epsilon_x
            self.list_q_IGM.append(q_k)

         return self.list_q_IGM[-1]

RobotModel.ComputeIGM = ComputeIGM

Compute $q^{\ast}$ when the function is called with the following arguments:

a) $X_d=X_{d_i}=\left(-0.1, -0.7, 0.3\right)^t$, $q_0=\left[ -1.57, 0.00 , -1.47, -1.47, -1.47 \right.$ $\left., -1.47 \right]$, $k_{max}=100$, $\epsilon_x=1$mm?

b) $X_d=X_{d_f}=\left(0.64 , -0.10, 1.14\right)^t$, $q_0=\left[ 0 , 0.80 , 0.00 , 1.00 , 2.00 , 0.00 \right]$, $k_{max}=100$, $\epsilon_x=1$mm?

Check the accuracy of the results using the function calculated in __Q3.__

In [50]:
k_max = 100
epsilon_x = 1

X_di = np.array([-0.10, -0.70, +0.30]).T
q_0i = np.array([-1.57, +0.00, -1.47, -1.47, -1.47, -1.47])

X_df = np.array([+0.64, -0.10, +1.14]).T
q_0f = np.array([+0.00, +0.80, +0.00, +1.00, +2.00, +0.00])

print(RobotTutorials.ComputeIGM(X_di, q_0i, k_max, epsilon_x))
print(RobotTutorials.ComputeIGM(X_df, q_0f, k_max, epsilon_x))

[ 9.28104782e+02  9.29674782e+02  5.15922497e+03 -1.47000000e+00
 -1.47000000e+00 -1.47000000e+00]
[0.  0.8 0.  1.  2.  0. ]


$q_{k+1}$ = q_{k} + J^{#}(q) \episilon_{x}

J^{#} = np.linalg.pinv(J)

\episilon_{k} = x_{1} - x_{k} = MGD(q_{k})

## IKM, Inverse Kinematic Model

In this question, the trajectory of the end-effector to be followed in the task space must allow the desired final position $X_{d_f}$ to be reached by following a straight line in the task space starting at the initial position $X_{d_i}$. This rectilinear motion is carried out at a constant speed $V=1m.s^{-1}$ and is sampled at a period $T_e=1$ms. The position of the end effector at the time instant $kT_e$ is noted $X_{d_k}$. The initial configuration of the robot is given by $q_i$ (found in question __Q4.__).

### Q8.

Using the inverse differential kinematic model, write a function entitled $ComputeIKM( \text{self}, X_{d_i}, X_{d_f}, V, T_e, q_i)$ realizing the coordinate transform to provide the series of setpoint values $q_{d_k}$ corresponding to the $X_{d_k}$ to the joint drivers. To do this, after having programmed the time law corresponding to the required motion, you can use the function developed in question __Q7__ capable of calculating the iterative MGI from the pseudo-inverse of the Jacobian matrix.

In [51]:
def ComputeIKM(self, X_d_i, X_d_f, V, Te, q_i, k_max, eps_x):
        """ 
        Computation of the Inverse differential Kinematic Model (IKM) making the coordinate transform to provide the series of setpoint values qdk corresponding to the Xdk to the joint drivers
        
        Inputs: 
           - Trajectory of the end effector to be followed in the task space between:
               - the initial position "X_d_i"
               - the desired final position "X_d_f" to be reached.
           - Rectilinear motion carried out :
               - at a constant speed "V"
               - sampled at a period "Te"
           - Initial configuration of the robot "q_i"
           - Number "k_max" of maximal iteration in the recursive algorithm (to be used with "self.ComputeIGM")
           - Norm of the tolerated Cartesian error "eps_x" (to be used with "self.ComputeIGM")
           
        Outputs: 
           - List "self.discreteTime" that defines the the sampled temporal series for each time step
           - List "self.list_X_d_k" of the intermediate Cartesian poses to be reached by the robot
           - List "self.list_q_dk" of the joint vectors computed at each iteration k of the recursive algorithm (series of setpoint values qdk corresponding to the Xdk to the joint drivers)
        """     
        
        

RobotModel.ComputeIKM = ComputeIKM 

Check that the successively reached positions of the end-effector is following the desired trajectory. To do so, you can plot the error between the sequence of positions reached by the end device and the position set points at each time step.

### Q9.

Plot the temporal evolution of the joint variables $q_1$ to $q_6$ calculated in the previous question. For each joint variable, graphically overlay the allowable extreme values corresponding to the joint limits:
\begin{align*}
q_{min} = \left[\begin{array}{c}{ -\pi , -\frac{\pi}{2} , -\pi , -\pi , -\frac{\pi}{2}, -\pi }\end{array}\right] 
\end{align*}
and
\begin{align*}
q_{max} = \left[\begin{array}{c}{ 0 , \frac{\pi}{2} , 0  ,\frac{\pi}{2}, \frac{\pi}{2} , \frac{\pi}{2} }\end{array}\right] 
\end{align*}

Comment on the evolution of the joint variables obtained previously.

### Q10.

In this question, we modify the algorithm developed in question __Q8__. We wish to take into account the distance of the values taken by the joint variables from their limits in the computation of the inverse kinematic model.

To do so, you will need to consider a secondary task aiming at keeping some distance from the articular stops $q_{min}$ and $q_{max}$. By the technique of the gradient projected into the null space of ${}^0J_v \left(q\right)$, you will consider minimizing the following potential function:
\begin{align*}
			H_{lim}\left(q\right) = \sum_{i=1}^{n} \left(\frac{q_i-\overline{q}_{i}}{q_{max}-q_{min}}\right)^2 \hspace{0.2cm} \text{where} \hspace{0.2cm} \overline{q}_{i} =  \frac{q_{max}+q_{min}}{2} 
\end{align*}

First, provide below the theoretical analytical solution for the joint variables to this problem.

Then, develop a new function $ComputeIKMlimits(\text{self}, X_{d_i}, X_{d_f}, V, T_e, q_i, q_{min}, q_{max})$ which implements the inverse kinematic model able to take into account the previous secondary task.

In [ ]:
def ComputeIKMlimits(self, X_d_i, X_d_f, V, Te, q_i, k_max, eps_x, q_min, q_max):    
        """ 
        Computation of the Inverse differential Kinematic Model (IKM) making the coordinate transform to provide the series of setpoint values qdk corresponding to the Xdk to the joint drivers while taking into account the joint limits
        
        Inputs: 
           - Trajectory of the end effector to be followed in the task space between:
               - the initial position "X_d_i"
               - the desired final position "X_d_f" to be reached.
           - Rectilinear motion carried out :
               - at a constant speed "V"
               - sampled at a period "Te"
           - Initial configuration of the robot "q_i"
           - Number "k_max" of maximal iteration in the recursive algorithm (to be used with "self.ComputeIGM")
           - Norm of the tolerated Cartesian error "eps_x" (to be used with "self.ComputeIGM")
           - Vector of lower bound of joint variable "q_min"
           - Vector of upper bound of joint variable "q_max"
           
        Outputs: 
           - List "self.list_q_dk_limits" of the joint vectors computed at each iteration k of the recursive algorithm (series of setpoint values qdk corresponding to the Xdk to the joint drivers)
           - List "self.list_X_d_k" of the intermediate Cartesian poses to be reached by the robot    
        """     
        
        
        
            
RobotModel.ComputeIKMlimits = ComputeIKMlimits

Plot the new temporal evolution of the joint variables $q_1$ to $q_6$ for the reference trajectory given in the question __Q9__.

Comment on the values taken by the joint variables.